In [51]:
# 랭체인의 단점이 버전에 따라 기존 모듈들의 변화가 심함
import os
from langchain_openai import ChatOpenAI
import urllib.request
import gradio as gr
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_classic.chains import LLMChain, SequentialChain, RetrievalQA

### 키값 불러오기

In [28]:
from dotenv import load_dotenv

load_dotenv()
openAPI_KEY = os.getenv('openAPI')

os.environ['OPENAI_API_KEY'] = openAPI_KEY

### 경제금융용어700선.pdf로드

In [29]:
loader = PyPDFLoader("경제금융용어700선.pdf")
texts = loader.load_and_split()
print("문서의 수: ",len(texts))

문서의 수:  366


### 샘플데이터 출력
texts는 아래와 같은 포맷을 따름
Document(page_content=‘내용’, metadata={‘source’: 파일명, ‘page’: 페이지 번호}) 본문에만 접근하고 싶다면 각 청크 (문자열 원소) 에 .page_content를 붙여서 호출하면 됩니다.

In [30]:
print(texts[15].page_content)

3
ㄱ 
신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통
계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 
부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .
가계처분가능소득 
가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축
으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 
지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 
정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 
보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 
체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal 
Gross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누어 계산한 
지표로 가계의 구매력을 가장 정확히 가늠해 볼 수 있는 소득지표이다 . 
 연관검색어 : 국민총소득 (GNI)
가교은행
청산 대상 금융기관의 자산, 부채를 임시로 넘겨받아 예금, 출금 등의 업무를 대행하고 
합병, 채권채무관계 조정 등 후속조치를 수행하는 은행을 말한다. 금융기관이 파산한 경우 
예금보험제도는 청산, 매각, 자산부채승계, 가교은행을 통한 인수 등의 수단을 통해 부실 
금융기관을 처리한다. 이 중 가교은행을 통한 인수방식은 파산은행의 처리에 상당한 시간이 
소요될 것으로 예상할 경우 사용되며 새로운 은행을 설립하여 자산, 부채를 포괄승계하도록 
하고, 인수 희망자를 물색하는 등의 조치를 취하게 된다. 가교은행을 이용한 처리방식은 
금융기관 파산에 따른 충격을 완화하고 시간적인 여유를 확보할 수 있는 것이 장점이다. 
임무를 마치면 정리대상 금융기관과 마찬가지로 없어지는 한시적 기관이다.
가동률
생산능력 대비 생산실적의 백분율(생산실적/생산능력×100)로

### 데이터전처리
1. 각 청크에 마다 전처리 수행
2. 여기서는 실제 어디서부터가 금융용어 설명인가. 교재에 보니 13부터라고 하네.
3. 12까지 제목과 목차인듯 하다.

In [31]:
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [32]:
texts = texts[13:]
print(texts[0].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [34]:
# 뒤에 있는 청크들 확인
print(texts[-2])

page_content='352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는 유럽의 대표 주가지수인 Euro

In [35]:
# 마지막 청크는 필요없다
texts = texts[:-1]
print(texts[-1].page_content)

352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는 유럽의 대표 주가지수인 Euro STOXX의 변동성 지수

### 임베딩 및 벡터 데이터베이스 적재

In [36]:
embedding = OpenAIEmbeddings(chunk_size=100) # 이렇게 함으로써 청크가 더 늘어날 수 있다
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding
)

In [ ]:
# 청크 개수 확인
print(vectordb._collection.count())

352


In [39]:
# 청크마다 속성 확인
for item in vectordb._collection.get():
    print(item)

ids
embeddings
documents
uris
included
data
metadatas


In [40]:
# 0번 청크 출력
documents = vectordb._collection.get()['documents']
print("청크의 개수: ", len(documents))
print("0번 청크: ")
print(documents[0])

청크의 개수:  352
0번 청크: 
1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


### 벡터DB에서 검색해서 찾는 서비스를 구현해 보자

In [46]:
retriever = vectordb.as_retriever(search_kwargs = {"k":2})
# docs =retriever.get_relevant_documents("비트코인이 궁금해") # 과거버전
docs = retriever.invoke("비트코인이 궁금해")
print("유사한 문서 개수: ", len(docs))
print()
print("첫번째 문서: ",docs[0])
print("#" * 100)
print("두번째 문서: ",docs[1])

유사한 문서 개수:  2

첫번째 문서:  page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다른 
속성을 지닌다. 한편 2017년 8월 1일 비트코인(classic bi

### GPT-4와 연결

In [62]:
# 프롬프트 생성, context와 question 이미 정의된 키워드이구나!!!!
template = """당신의 이름은  금융쟁이 입니다 . 주님이 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate.from_template(template)

In [63]:
# llm gpt객체 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

LLM 객체인 llm, 프롬프트 템플릿 객체인 prompt, 검색기 객체인 retriever를 연결합니다. 

In [64]:
# 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt":prompt},
    retriever=retriever,
    return_source_documents=True
)

### 테스트

In [65]:
input_text = "디커플링이란 무엇인가?"

chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '디커플링이란 무엇인가?', 'result': '디커플링은 탈동조화라고도 하는데, 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 말해. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 미국 경제와 다른 모습을 보이는 경우가 있어. 주가나 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 흐름을 보이는 현상도 디커플링이라고 할 수 있어. 반대로, 경제가 다른 국가나 세계 경제 흐름과 유사한 흐름을 보이는 것을 커플링이라고 해.', 'source_documents': [Document(metadata={'creationdate': 'January 31, 2018 16:21', 'page_label': '115', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'moddate': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'page': 114, 'source': '경제금융용어700선.pdf', 'creationdate--text': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'author': 'INSUNG DATA INC.', 'total_pages': 371, 'moddate--text': 'January 31, 2018 16:21'}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통화정책\n디커플링/커플링\n

In [66]:
# 결과값만 뽑아내는 함수를 만들자. 위에 결과는 너무 복잡하다.
def get_chatbot_response(input_text):
    chatbot_response = qa_chain(input_text)
    return chatbot_response['result'].strip()

In [67]:
input_text = "너는 누가 만들었니?"
result = get_chatbot_response(input_text)
print(result)

검색 결과에 따르면, 나는 주님이 만들었다고 하네.
